<a href="https://colab.research.google.com/github/RodolfoFigueroa/madi2022-1/blob/main/Unidad_3/4_Backtracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [90]:
import numpy as np

En esta práctica veremos varios ejemplos de exploraciones exhaustivas, es decir, ir encontrando en el espacio de búsqueda elementos que cumplen ciertas características deseadas, una forma común de hacer esto es utilizando la técnica de backtracking (o vuelta hacia atrás), en donde se van buscando elementos con determinadas características en cada paso.

# Ejemplos

## Cambio
Recordando un poco el ejemplo que vimos anteriormente sobre monedas y sus denominaciones, consideremos una lista $L$ de denominaciones posibles para nuestras monedas, y nuestro problema ahora será determinar de cuántas formas se puede formar el número $n$ utilizando dichas denominaciones (suponiendo que tenemos tantas monedas de cada denominación como sea necesario).

Dada la lista $L$, podemos hacer backtracking del siguiente modo:

* Iteramos sobre los elementos de la lista, y definimos una función `count(i,k)` que nos dice de cuántas formas podemos sumar a $k$ con denominaciones $L[i], L[i+1], \dots$.
* Podemos calcular el valor de `count(i,k)` de manera recursiva del siguiente modo:
    * Si $L[i] > k$, esto nos dice que la moneda actual (`L[i]`) es más grande que el dinero que queremos partir (`k`), así que nos movemos a la siguiente, i.e. `count(i,k) = count(i+1, k)`.
    * Si $L[i] \leq k$, entonces podemos cambiar el dinero con la moneda actual. Entonces, consideramos dos casos: uno en el que sí lo cambiamos y nos quedamos en la misma moneda (`count(i, k-L[i])`) y otro en el que no lo hacemos y simplemente pasamos a la siguiente moneda (`count(i+1, k)`).

Veamos una implementación de este algoritmo:

In [12]:
combs = []

def count(L, k, i=0, out=None):
    if out is None:
        out = []
    if i >= len(L):
        return 0
    if k == 0:
        #print('entro', out)
        combs.append(out)
        return 1
    if L[i] > k:
        return count(L, k, i+1, out)
    else:
        temp = out.copy()
        temp.append(L[i])
        #print('temp', temp)
        return count(L, k-L[i], i, temp) + count(L, k, i+1, out)

L = [2, 8, 4, 6]
print(count(L, 10))
print(combs)

6
[[2, 2, 2, 2, 2], [2, 2, 2, 4], [2, 2, 6], [2, 8], [2, 4, 4], [4, 6]]


## Reinas

Un famoso problema algorítmico es que dado un tablero de ajedrez (de $8 \times 8$), se determine cuántas formas hay de colocar $8$ reinas en el tablero de tal manera que no haya dos que se ataquen entre sí. Este es un ejemplo clásico de la técnica de backtracking, resolveremos el problema para un tablero de $n\times n$ con $n$ reinas.

Primero, representaremos el tablero como una lista $Q$ de tamaño $n$. La entrada $Q_i$ será un número $0\leq j < n$ que nos dirá la columna en la cual se encuentra la reina de la $i$-ésima fila (de abajo hacia arriba). Por ejemplo, para el siguiente arreglo de reinas:

![chess](chess.png)

Su representación sería `[1, 3, 0, 2]`.

Luego, definimos la siguiente función:

In [66]:
def place_queens(n, row=0, Q=None, queens=None):
    if Q is None:
        Q = [None] * n
    if queens is None:
        queens = []
        
    if row == n:
        queens.append(Q)
        return
    
    for col in range(n):
        legal = True
        for j in range(row):
            if Q[j] == col or Q[j] == col+row-j or Q[j] == col-row+j:
                legal = False
                break
        if legal:
            Q[row] = col
            place_queens(n, row+1, Q.copy(), queens)
    return queens

Esta acepta cuatro argumentos:
* `n`: Tamaño del tablero
* `row`: Fila a partir de la cual se intenta insertar reinas, después de la cual se sube recursivamente hasta llegar a la última. Por defecto es cero (i.e., llenar el tablero completo)
* `Q`: Lista de posiciones de las reinas, en el formato descrito previamente. Al principio está lleno de `None`.
* `queens`: Lista de todas las soluciones posibles. Al principio es vacía.

El algoritmo consiste en ir insertando reinas recursivamente, empezando de la fila de abajo. Se utiliza un bucle para intentar insertarla en cada columna de la fila actual, checando todas las filas anteriores para ver si la posición es válida. Si sí lo es, se inserta y se pasa a la siguiente.

Lo intentamos con un tablero de $4\times 4$:

In [69]:
queens = place_queens(4)
queens

[[1, 3, 0, 2], [2, 0, 3, 1]]

## Suma de subconjuntos

Dado un conjunto de números positivos $S$ y un número $x$, queremos determinar si existe un subconjunto $U\subseteq S$ tal que la suma de los elementos de $U$ es $x$.

Primero, notemos que hay dos casos base: si $x$ es cero, regresamos `verdadero` inmediatamente, ya que la suma del conjunto vacío lo cumple. Por otro lado, si $x<0$, o si $x\neq 0$ pero $S$ es vacío, regresamos `falso`, ya que no existe solución.

Por otro lado, para el caso general, consideremos un elemento $s\in S$ arbitrario. Existe un subconjunto de $S$ que suma a $x$ si y solo si alguna de las dos proposiciones siguientes es verdadera:

* Existe un subconjunto $U\subseteq S$ que suma a $x$ y $s\in U$.
* Existe un subconjunto $U\subseteq S$ que suma a $x$ y $s\notin U$.

El primer caso implica que debe de existir un subconjunto de $S$ que no incluya a $s$ y que sume a $x-s$. En el segundo caso, debe de existir un subconjunto de $S$ que no incluya a $s$, y que sume a $x$. Con esto, podemos reducir el problema a los siguientes dos subproblemas:

* Subconjunto de $S\setminus \{s\}$ que sume a $x-s$.
* Subconjunto de $S\setminus \{s\}$ que sume a $x$.

Así, podemos definir el algoritmo recursivo:

In [3]:
def subset_sum(S, x):
    if x == 0:
        return True
    elif len(S) == 0 or x < 0:
        return False
    
    S_minus = S.copy()
    s = S_minus.pop()
    b1 = subset_sum(S_minus, x-s)
    b2 = subset_sum(S_minus, x)
    
    return b1 or b2

Probándolo en un conjunto que sí sabemos que funciona:

In [5]:
S = [1,3,8,4]
x = 15
subset_sum(S, x)

True

Y en uno que no funciona:

In [6]:
S = [1,3,8,4]
x = 2
subset_sum(S, x)

False

Hacer copias de un arreglo es una operación costosa. Es más eficiente pasar el mismo arreglo cada vez, y simplemente cambiar los índices que consideramos:

In [83]:
def subset_sum_index(S, x, r=None):
    if r is None:
        r = len(S)
        
    if x == 0:
        return True
    elif r == 0 or x < 0:
        return False
    
    s = S[r-1]
    b1 = subset_sum_index(S, x-s, r=r-1)
    b2 = subset_sum_index(S, x, r=r-1)
    
    return b1 or b2

In [85]:
S = [1,3,8,4]
x = 15
subset_sum_index(S, x)

True

Comparando los tiempos de ejcución para una lista de enteros grande:

In [102]:
big_list = list(np.random.randint(0, 100, 300))

In [103]:
%%timeit
subset_sum(big_list, x)

5.12 s ± 196 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [104]:
%%timeit
subset_sum_index(big_list, x)

4.66 s ± 128 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Ejercicios

## Ejercicio 1
Dado un conjunto de palabras y un string, describe e implementa un algoritmo que permita contar cuántas oraciones diferentes puede formar la string en cuestión suponiendo que las únicas palabras que existen son las del conjunto inicial. Por ejemplo, si el conjunto de palabras es $\{hola, ola, h\}$, la string "holah" tiene dos posibles interpretaciones, una es "hola h", y la segunda "h ola h". Verifica tu algoritmo con: 

*   Conjunto de palabras `{a, as, tin, tinar, san, sana, atina, arce, ce, atinar}`, y la string "atinarcesanas".
*   Conjunto de palabras `{i, like, ice, and, cream, icecream, man, go, mango}` y la string "ilikeicecreamandmango".

**Descripción de tu algoritmo**

Primero definiremos el algoritmo para encontar todas las segmentaciones posible de una palabra dada. Para eso utilizaré backtracking, primero veremos como estara formado nuestro árbol de recursion para visualizar en que momento haremos backtrack. 
Vemos que para encontrar las posibles maneras de segmentacion, tendremos que empezar el albol con las segmentacion de $1$ hasta $n$ en el primer nivel, por ejemplo:

```
string = 'holamundo'

                   raiz
 /   /   /      /         \     \           \       
 h ho hol hola holam holamun holamund holamundo
/\ ...                ...                   ...

```
Una vez encontrado el primer nivel, podemos ahora entender mejor el algortimo, el segundo nivel consistira nuevamente con las siguiente palabras con longitud que va variando en uno en uno. Es decir, tendremos como nodos las siguientes secuencia de caracteres tal que su longitud va a hacer siempre una incremento mas hasta llegar al ultimo caracter:

```
numero (n) sera igual a la longitud(particion)
                   
                    node
   /    /    /      /     \    \      \       
   1    2    3   4         5     6   ... n 

```

Con esto genera un arbol con las posibles segmentaciones y lo que tenemos que hacer es ir caminando sobre las ramas hasta llegar a lo hoja, que será cuando ya no sea posible encontrar más segmentaciones, en nuestro caso cuando la variable $i$ sea mayor a la longitud de la palabra, una vez llegado a este punto guardamos esa segmentación si solo si cumple la condición de que pertenezca al conjunto, en caso contrario, solo hacemo paso hacia atras para ir buscando nuevamente las posibles segmentaciones hasta llegar a la hoja de nuestro árbol.

In [284]:

def count_word(word, a=[],i=0, sols=0, conj=None):
    """
    Cuenta cuántas oraciones diferentes puede formar la string en cuestión suponiendo
    que las únicas palabras que existen son las del conjunto inicial
    """           
    
    if i >= len(word):
        # La segmentacion esta completa       
        flag = True
        # Verificamos que este cada segmento en el conjunto
        for seg in a:            
            if seg not in conj:
                flag = False
                break
                
        if flag is True:                            
            sols += 1
            print('sol:', a)
        return sols
    
    candidatos = [word[i:j] for j in range(i+1, len(word)+1)]
    for k, c in enumerate(candidatos):             
        a.append(c)
        sols = count_word(word, a, i+(k+1), sols, conj)
        a.remove(c)
    return sols    

## Tests

In [285]:
c3 = ['hola', 'ola', 'h']
string3 = 'holah'
count_word(string3, conj=c3)

sol: ['h', 'ola', 'h']
sol: ['hola', 'h']


2

In [282]:
c1 = ['a', 'as', 'tin', 'tinar', 'san', 'sana', 'atina', 'arce', 'ce', 'atinar']
string1 = 'atinarcesanas'
print('Numero de oraciones diferentes:', count_word(string1, conj=c1))

Numero de oraciones diferentes: 3


In [283]:
c2 = ['i', 'like', 'ice', 'and', 'cream', 'icecream', 'man', 'go', 'mango']
string2 = 'ilikeicecreamandmango'
print('Numero de oraciones diferentes:', count_word(string2, conj=c2))

Numero de oraciones diferentes: 4


## Ejercicio 2
Supón que ahora en el ejemplo 1 no se tienen tantas monedas como se deseen. Es decir, se tiene una lista $L$ de denominaciones posibles, y un entero $k$ que nos indica que tenemos exactamente $k$ monedas de cada denominación posible. Describe e implementa un algoritmo que permita contar de cuántas formas se puede formar un entero $n$ con monedas de las denominaciones dadas, y usando a lo más $k$ monedas de cada denominación.

**Descripción de tu algoritmo**

Para abordar primero el problema debemos pensar en como va ser la entrada y su estructura para después modelar nuestro algortimo. Ya que ahora será monedas finitas podemos pensar en una _lista_ que nos indique exactamente cuantas monedas tenemos y se indicará con la repeticion de la misma denominación, por ejemplo:

$$L = [2, 2, 2, 4, 6, 4, 10]$$

Nuestra lista nos dice que tenemos solamente tres $2$ para gastar dos $4$, un $6$ y por último un $10$. 

Una vez definida nuestra entrada, podemos proseguir con nuestro algoritmo que tomará esta entrada y nos devolvera la cantidad de formas para formar un entero $n$.

Basicamente será backtracking, primero definimos nuestros casos base donde el algoritmo entrará para terminar la recursión. 
Nuestro algortimo empieza primero en verificar si el primer número de la lista es menor al valor buscado $n$, si es así tendremos dos casos, el primero va considerando que hemos deseado llevar la cuenta con la moneda que cumplio la condicion y generará una nueva lista quitando la moneda seleccionada, esto genera un camino donde llevaremos la cuenta hasta llegar a que $n=0$ que indicara que nuestras monedas que hemos encontrado son las suficientes para formar $n$ y terminará la recursión. Por último el segundo caso contempla seguir con la siguiente moneda y verificar si con ella podemos iniciar un posible camino que nos llevara a ver si podemos formar $n$ y esto se hará recursivamente hasta revisar las monedas.

In [79]:
def count(L, n, i=0, comb=None):    
    """Regresa la cantidad de formas de formar un entero n"""
    if comb is None:
        comb = []
        
    if n == 0:
        print('comb',comb)
        return 1
        
    if i >=  len(L):
        return 0    
    
    if L[i] > n:        
        return count(L, n, i+1, comb)
    else:          
        aux = comb.copy()
        aux.append(L[i])
        new_L = L.copy()
        new_L.remove(L[i])
        return count(new_L, n-L[i], i, aux) + count(L,n, i+1, comb)

### Test 1

Usamos la misma lista que el caso anterior de monedas infinitas, vemos que ahora soló nos da 2 debido a que nuestra monedas ahora estan contadas, solo podemos tomas un 2 y 6 para dar 10 y un 5 y 6 respectivamente.

In [80]:
L = [2, 8, 4, 6]
print('Cantidad de formas:', count(L, 10))

comb [2, 8]
comb [4, 6]
Cantidad de formas: 2


### Test 2

Podemos ver que la correctud el algoritmo, ya que nos muestra 8 formas para formar 12 unidades, vemos que nos apareces 4 veces el arreglo $[2,4,6]$ debido que le primer 2 es tomado de la primera modena y se toma el primer 4 y luego el 6, despues para la segunda se toma el primer 2, el segundo 4 y luego 6, despues se hace lo mismo, pero ahora con el segundo 2, primer 4 y luego 6.Y por último se toma el seguno 2, segundo 4 y luego 6.

In [75]:
L = [2, 2, 4, 8, 4, 6]
print('Cantidad de formas:', count(L, 12))

comb [2, 2, 4, 4]
comb [2, 2, 8]
comb [2, 4, 6]
comb [2, 4, 6]
comb [2, 4, 6]
comb [2, 4, 6]
comb [4, 8]
comb [8, 4]
Cantidad de formas: 8
